In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import train_test_split, RandomizedSearchCV, StratifiedKFold
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix,roc_auc_score,roc_curve
from sklearn.tree import  DecisionTreeClassifier
from sklearn import svm
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_selection import RFE
from xgboost import XGBClassifier
from sklearn.model_selection import KFold,cross_val_score

In [6]:
df1 = pd.read_csv("Preprocessed_Bondora.csv")

In [7]:
df1

,NewCreditCustomer,VerificationType,Age,Gender,Country,AppliedAmount,Amount,Interest,LoanDuration,MonthlyPayment,...,HomeOwnershipType,IncomeTotal,LiabilitiesTotal,DebtToIncome,PrincipalBalance,InterestAndPenaltyBalance,AmountOfPreviousLoansBeforeLoan,PreviousRepaymentsBeforeLoan,PreviousEarlyRepaymentsCountBeforeLoan,Target
0,1,1,61,0,0,319.5582,115.0408,30.00,12,0.00,...,6,10500.0,0.00,0.00,0.00,0.00,83.0852,0.0000,0,0
1,0,1,48,0,0,191.7349,140.6057,25.00,1,0.00,...,6,10800.0,0.00,0.00,0.00,0.00,255.6467,258.6256,0,0
2,1,1,58,0,0,319.5582,319.5409,25.00,20,0.00,...,6,7000.0,0.00,0.00,116.35,414.07,0.0000,0.0000,0,1
3,1,1,23,0,0,127.8233,57.5205,45.00,15,0.00,...,6,11600.0,0.00,0.00,0.00,0.00,134.2144,0.0000,0,0
4,1,1,25,0,0,319.5582,319.5436,30.00,12,0.00,...,6,6800.0,0.00,0.00,0.00,0.00,146.9966,0.0000,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76819,1,2,37,1,2,3000.0000,3000.0000,31.01,60,107.68,...,0,1400.0,500.00,7.69,0.00,0.00,0.0000,0.0000,0,1
76820,0,2,35,1,2,3000.0000,3000.0000,18.25,60,85.33,...,8,2265.0,1777.46,31.58,0.00,0.00,3000.0000,389.3600,0,0
76821,1,0,40,1,2,2500.0000,2500.0000,24.83,60,80.42,...,8,2500.0,1350.00,3.22,630.24,1227.98,0.0000,0.0000,0,1
76822,0,3,47,0,0,3000.0000,3000.0000,17.74,60,84.51,...,11,540.0,732.05,44.35,0.00,0.00,500.0000,96.8000,0,1


In [8]:
X = df1.drop(columns = ["Target", "AppliedAmount"])
Y = df1['Target']

In [9]:
Y.value_counts()

1    42383
0    34441
Name: Target, dtype: int64

In [10]:
params={
 "learning_rate"    : [0.05, 0.10, 0.15, 0.20, 0.25, 0.30 ] ,
 "max_depth"        : [ 3, 4, 5, 6, 8, 10, 12, 15, 18, 20],
 "min_child_weight" : [ 1, 3, 5, 7 ],
 "gamma"            : [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
 "colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7 ]
    
}

In [ ]:
xgb_clf=XGBClassifier()
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
random_search=RandomizedSearchCV(xgb_clf, param_distributions=params, n_iter=5, scoring='roc_auc',
                                 n_jobs=-1, cv=kfold, verbose=3, random_state=42)
random_search.fit(X, Y)


Fitting 10 folds for each of 5 candidates, totalling 50 fits


In [ ]:
random_search.best_params_

In [ ]:
xgb_clf_imb = XGBClassifier(min_child_weight=5, max_depth = 12, learning_rate=0.15,gamma= 0.3,
 colsample_bytree=0.7)
xgb_clf_imb.fit(X, Y)
predictions = xgb_clf_imb.predict(X)

print("Confusion Matrix:")
print(confusion_matrix(Y, predictions))

print("Classification Report")
print(classification_report(Y, predictions))

In [ ]:
xgb_clf_imb.score(X, Y)

In [ ]:
df1.corr().sort_values("Target")["Target"]

In [ ]:
rfe = RFE(xgb_clf_imb, n_features_to_select = 12)
fit = rfe.fit(X, Y)

fit.ranking_

In [ ]:
data = pd.DataFrame({"Column":X.columns,"Value":fit.ranking_})

data.sort_values("Value")

In [ ]:
AmountOfPreviousLoansBeforeLoan          -0.073826
PreviousRepaymentsBeforeLoan             -0.068554
EmploymentDurationCurrentEmployer        -0.021301
UseOfLoan                                -0.020796
IncomeTotal                              -0.016028
VerificationType                         -0.015009
PreviousEarlyRepaymentsCountBeforeLoan   -0.013581
OccupationArea                           -0.010468
LiabilitiesTotal                         -0.002728
Education                                 0.018500
HomeOwnershipType                         0.022476
MaritalStatus                             0.027862
EmploymentStatus                          0.028930
Age                                       0.047144
Amount                                    0.048753
Gender                                    0.056514
NewCreditCustomer                         0.067275
DebtToIncome                              0.081891


In [28]:
keep = ['Country','Amount', 'Interest', 'LoanDuration', 'MonthlyPayment', 'UseOfLoan',
       'MaritalStatus', 'EmploymentStatus',
       'DebtToIncome','PrincipalBalance', 'InterestAndPenaltyBalance',
       'PreviousRepaymentsBeforeLoan','NewCreditCustomer','Target']

In [39]:
df = pd.read_csv("Bondora_preprocessed.csv")

In [40]:
new = df[keep]

In [41]:
new

,Country,Amount,Interest,LoanDuration,MonthlyPayment,UseOfLoan,MaritalStatus,EmploymentStatus,DebtToIncome,PrincipalBalance,InterestAndPenaltyBalance,PreviousRepaymentsBeforeLoan,NewCreditCustomer,Target
0,EE,115.0408,30.00,12,0.00,Other,Married,Fully-Employed,0.00,0.00,0.00,0.0000,True,0
1,EE,140.6057,25.00,1,0.00,Other,Divorced,Fully-Employed,0.00,0.00,0.00,258.6256,False,0
2,EE,319.5409,25.00,20,0.00,Home Improvement,Married,Fully-Employed,0.00,116.35,414.07,0.0000,True,1
3,EE,57.5205,45.00,15,0.00,Loan Consolidation,Single,Not_specified,0.00,0.00,0.00,0.0000,True,0
4,EE,319.5436,30.00,12,0.00,Vehicle,Cohabitant,Fully-Employed,0.00,0.00,0.00,0.0000,True,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77389,FI,3000.0000,31.01,60,107.68,Loan Consolidation,Single,Fully-Employed,7.69,0.00,0.00,0.0000,True,1
77390,FI,3000.0000,18.25,60,85.33,Home Improvement,Single,Fully-Employed,31.58,0.00,0.00,389.3600,False,0
77391,FI,2500.0000,24.83,60,80.42,Loan Consolidation,Married,Entrepreneur,3.22,630.24,1227.98,0.0000,True,1
77392,EE,3000.0000,17.74,60,84.51,Other,Married,Fully-Employed,44.35,0.00,0.00,96.8000,False,1


In [42]:
new.isnull().sum()

Country                         0
Amount                          0
Interest                        0
LoanDuration                    0
MonthlyPayment                  0
UseOfLoan                       0
MaritalStatus                   0
EmploymentStatus                0
DebtToIncome                    0
PrincipalBalance                0
InterestAndPenaltyBalance       0
PreviousRepaymentsBeforeLoan    0
NewCreditCustomer               0
Target                          0
dtype: int64

In [43]:
new.Country.value_counts()

EE    39896
FI    20677
ES    16527
SK      294
Name: Country, dtype: int64

In [44]:
new = new[new.Country != 'SK']

In [45]:
new.Country.value_counts()

EE    39896
FI    20677
ES    16527
Name: Country, dtype: int64

In [46]:
new.NewCreditCustomer.value_counts()

True     48580
False    28520
Name: NewCreditCustomer, dtype: int64

In [47]:
new.Target.value_counts()

1    42525
0    34575
Name: Target, dtype: int64

In [48]:
new.Target.value_counts()

1    42525
0    34575
Name: Target, dtype: int64

In [49]:
new.to_csv("Modelling_data.csv")

In [2]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn_pandas import DataFrameMapper

In [3]:
bon = pd.read_csv("Modelling_data.csv")
bon.drop(columns='Unnamed: 0',inplace=True)
bon

,Country,Amount,Interest,LoanDuration,MonthlyPayment,UseOfLoan,MaritalStatus,EmploymentStatus,DebtToIncome,PrincipalBalance,InterestAndPenaltyBalance,PreviousRepaymentsBeforeLoan,NewCreditCustomer,Target
0,EE,115.0408,30.00,12,0.00,Other,Married,Fully-Employed,0.00,0.00,0.00,0.0000,True,0
1,EE,140.6057,25.00,1,0.00,Other,Divorced,Fully-Employed,0.00,0.00,0.00,258.6256,False,0
2,EE,319.5409,25.00,20,0.00,Home Improvement,Married,Fully-Employed,0.00,116.35,414.07,0.0000,True,1
3,EE,57.5205,45.00,15,0.00,Loan Consolidation,Single,Not_specified,0.00,0.00,0.00,0.0000,True,0
4,EE,319.5436,30.00,12,0.00,Vehicle,Cohabitant,Fully-Employed,0.00,0.00,0.00,0.0000,True,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77095,FI,3000.0000,31.01,60,107.68,Loan Consolidation,Single,Fully-Employed,7.69,0.00,0.00,0.0000,True,1
77096,FI,3000.0000,18.25,60,85.33,Home Improvement,Single,Fully-Employed,31.58,0.00,0.00,389.3600,False,0
77097,FI,2500.0000,24.83,60,80.42,Loan Consolidation,Married,Entrepreneur,3.22,630.24,1227.98,0.0000,True,1
77098,EE,3000.0000,17.74,60,84.51,Other,Married,Fully-Employed,44.35,0.00,0.00,96.8000,False,1


In [4]:
bon.NewCreditCustomer.value_counts()

True     48580
False    28520
Name: NewCreditCustomer, dtype: int64

In [5]:
bon.describe()

,Amount,Interest,LoanDuration,MonthlyPayment,DebtToIncome,PrincipalBalance,InterestAndPenaltyBalance,PreviousRepaymentsBeforeLoan,Target
count,77100.000000,77100.000000,77100.000000,77100.000000,77100.000000,77100.000000,77100.000000,77100.000000,77100.000000
mean,2472.418571,40.346683,44.412646,119.655540,11.972109,1370.019173,1180.962873,648.617883,0.551556
std,2164.356324,30.666013,16.415282,130.967402,18.841391,1943.612441,3111.557320,1840.327433,0.497338
min,6.390000,3.000000,1.000000,0.000000,0.000000,-34.200000,-2.660000,0.000000,0.000000
25%,740.000000,23.380000,36.000000,25.440000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2020.000000,32.000000,48.000000,90.340000,0.000000,515.375000,83.225000,0.000000,1.000000
75%,3185.000000,50.450000,60.000000,162.920000,20.710000,2111.890000,1064.000000,371.750000,1.000000
max,10632.000000,264.310000,60.000000,2368.540000,198.020000,10632.000000,64494.770000,34077.420000,1.000000


In [6]:
X = bon.drop("Target",axis=1)
y = bon["Target"]

In [8]:
y

0        0
1        0
2        1
3        0
4        1
        ..
77095    1
77096    0
77097    1
77098    1
77099    1
Name: Target, Length: 77100, dtype: int64

In [9]:
m = DataFrameMapper([(["Country",LabelEncoder()]),(["UseOfLoan",LabelEncoder()]),
                     (["MaritalStatus",LabelEncoder()]),
                     (["EmploymentStatus",LabelEncoder()])
                    ])

In [10]:
classifier = []
classifier.append(("mapper",m))
classifier.append(("model", XGBClassifier(min_child_weight=1, max_depth = 12,learning_rate=0.2,gamma= 0.3,
 colsample_bytree=0.5)))

In [11]:
classifier

[('mapper',
  DataFrameMapper(drop_cols=[],
                  features=[['Country', LabelEncoder()],
                            ['UseOfLoan', LabelEncoder()],
                            ['MaritalStatus', LabelEncoder()],
                            ['EmploymentStatus', LabelEncoder()]])),
 ('model',
  XGBClassifier(base_score=None, booster=None, callbacks=None,
                colsample_bylevel=None, colsample_bynode=None,
                colsample_bytree=0.5, early_stopping_rounds=None,
                enable_categorical=False, eval_metric=None, gamma=0.3,
                gpu_id=None, grow_policy=None, importance_type=None,
                interaction_constraints=None, learning_rate=0.2, max_bin=None,
                max_cat_to_onehot=None, max_delta_step=None, max_depth=12,
                max_leaves=None, min_child_weight=1, missing=nan,
                monotone_constraints=None, n_estimators=100, n_jobs=None,
                num_parallel_tree=None, predictor=None, random_state=No

In [12]:
model= Pipeline(classifier)

In [13]:
model

Pipeline(steps=[('mapper',
                 DataFrameMapper(drop_cols=[],
                                 features=[['Country', LabelEncoder()],
                                           ['UseOfLoan', LabelEncoder()],
                                           ['MaritalStatus', LabelEncoder()],
                                           ['EmploymentStatus',
                                            LabelEncoder()]])),
                ('model',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=None, colsample_bynode=None,
                               colsample_bytree=0.5, early_stopping_rounds=No...
                               gamma=0.3, gpu_id=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=0.2,
                               max_bin=None, max_cat_to_onehot=None,
                               max_delta_step=None, max_depth=12,
                               max_leaves=None, min_child_weight=1, missing=nan,
                               monotone_constraints=None, n_estimators=100,
                               n_jobs=None, num_parallel_tree=None,
                               predictor=None, random_state=None,
                               reg_alpha=None, reg_lambda=None, ...))])

In [14]:
model.fit(X,y)

Pipeline(steps=[('mapper',
                 DataFrameMapper(drop_cols=[],
                                 features=[['Country', LabelEncoder()],
                                           ['UseOfLoan', LabelEncoder()],
                                           ['MaritalStatus', LabelEncoder()],
                                           ['EmploymentStatus',
                                            LabelEncoder()]])),
                ('model',
                 XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=0.5, early_stopping_rounds=None,...
                               gamma=0.3, gpu_id=-1, grow_policy='depthwise',
                               importance_type=None, interaction_constraints='',
                               learning_rate=0.2, max_bin=256,
                               max_cat_to_onehot=4, max_delta_step=0,
                               max_depth=12, max_leaves=0, min_child_weight=1,
                               missing=nan, monotone_constraints='()',
                               n_estimators=100, n_jobs=0, num_parallel_tree=1,
                               predictor='auto', random_state=0, reg_alpha=0,
                               reg_lambda=1, ...))])

In [23]:
bon[8:10]

,Country,Amount,Interest,LoanDuration,MonthlyPayment,UseOfLoan,MaritalStatus,EmploymentStatus,DebtToIncome,PrincipalBalance,InterestAndPenaltyBalance,PreviousRepaymentsBeforeLoan,NewCreditCustomer,Target
8,EE,31.9498,20.0,12,0.0,Home Improvement,Single,Fully-Employed,0.0,0.0,0.0,0.0,True,1
9,EE,319.5583,25.0,12,0.0,Loan Consolidation,Married,Fully-Employed,0.0,0.0,0.0,0.0,True,0


In [24]:
model.predict(X[8:10])

array([1, 0])

In [61]:
import pickle

In [62]:
with open(file="final_model.pkl",mode="wb") as f:
    pickle.dump(model,f)